# Deploy with Docker 

The simplest way to deploy a Vespa app.

## Deploy application package created with pyvespa

This section assumes you have an [ApplicationPackage](reference-api.rst#vespa.package.ApplicationPackage) 
instance assigned to `app_package` containing your app desired configuration.
For the purpose of this demonstration we are going to use a minimal (and useless) application package:

We can locally deploy our `app_package` using Docker without leaving the notebook, by creating an instance of [VespaDocker](reference-api.rst#vespa.deployment.VespaDocker), as shown below:

`app` now holds a [Vespa](reference-api.rst#vespa.application.Vespa) instance, which we are going to use to interact with our application. Congratulations, you now have a Vespa application up and running.

## Learn Vespa by looking at underlying config files

It is important to know that `pyvespa` simply provides a convenient API to define Vespa application packages from python. `vespa_docker.deploy` export Vespa configuration files to the `disk_folder` defined above. Going through those files is a nice way to start learning about Vespa syntax.

It is also possible to export the Vespa configuration files representing an application package created with `pyvespa` without deploying the application by using the `export_application_package` method: 

This will export the application files to an `application` folder within the `disk_folder`.

## Deploy application package from config files

The `pyvespa` API provides a subset of the functionality available in Vespa. The reason is that `pyvespa` is meant to be used as an experimentation tool for Information Retrieval (IR) and not for building production-ready applications. So, the python API expands based on the needs to replicate common use cases that often require IR experimentation.

If the application requires functionality or fine-tuning not available in `pyvespa`, simply build it directly using Vespa configuration files as shown in [many examples](https://docs.vespa.ai/en/getting-started.html) on Vespa docs. But even in this case, one can still get value out of `pyvespa` by deploying it from python based on the Vespa configuration files stored on disk.

Make sure Docker is running with minimum 4G:

In [11]:
!docker info | grep "Total Memory"

 Total Memory: 15.64GiB


Clone and deploy the news search app covered in this [Vespa tutorial](https://docs.vespa.ai/en/tutorials/news-3-searching.html):

In [12]:
!cd $WORK_DIR
!git clone --depth 1 https://github.com/vespa-engine/sample-apps.git

Cloning into 'sample-apps'...
remote: Enumerating objects: 1138, done.
remote: Counting objects: 100% (1138/1138), done.
remote: Compressing objects: 100% (754/754), done.
remote: Total 1138 (delta 189), reused 888 (delta 109), pack-reused 0
Receiving objects: 100% (1138/1138), 18.94 MiB | 14.91 MiB/s, done.
Resolving deltas: 100% (189/189), done.


The Vespa configuration files of the news search app are stored in `sample-apps/news/app-3-searching/`:

In [14]:
!tree sample-apps/news/app-3-searching/

sample-apps/news/app-3-searching/
├── schemas
│   └── news.sd
└── services.xml

1 directory, 2 files


Deploy to a Docker container from disk.
Note that `disk_folder` must be an absolute path, for Docker to bind the volume:

In [16]:
import os
from vespa.deployment import VespaDocker

vespa_docker_news = VespaDocker(
    disk_folder=os.getenv("WORK_DIR") + "/sample-apps/news/app-3-searching/",
    port=8090
)
app = vespa_docker_news.deploy_from_disk(application_name="news")

Waited for configuration server, 0/30 seconds...
Waited for configuration server, 5/30 seconds...


Waiting for application status.


Finished deployment.


`app` can now be used to feed and query the application just deployed. Clean up after use:

In [10]:
from shutil import rmtree

rmtree("sample-apps", ignore_errors=True)
vespa_docker_news.container.stop()
vespa_docker_news.container.remove()